In [4]:
import csv
from bs4 import BeautifulSoup

html_doc = open("Scraping/SDNY FULL_2FLY.htm")
# html_doc = open("Scraping/SDNY CM_ECF Version 6.1.1.htm")


soup = BeautifulSoup(html_doc, 'html.parser')

tr_list = soup.findAll('tr')

print(len(tr_list))

for thing in tr_list:
    u_list = thing.find_all('u')
    for u in u_list:
        print(u.getText())

841
Defendant                                 (1)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                                 (2)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                                 (3)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                                 (4)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                                 (5)
Pending Counts
Disposition
Highest Offense Level (Opening)
Terminated Counts
Disposition
Highest Offense Level (Terminated)
Complaints
Disposition
Defendant                      

In [16]:
# Create a flag to determine which text block
def iterate_through_html():
    
    which_block = None
    pending_counts = []
    terminated_counts = []
    
    underlined_text = ""
    
    for i in range(len(tr_list)):
        tr = tr_list[i]
        
        u = tr.find('u')
        
        title = False
        if u != None:
            title = True
        
        if title:
            underlined_text = u.getText()

        if title and "Defendant" in underlined_text:
            which_block = None
            
            print(underlined_text)

            # Get information from name block
            if i + 1 >= len(tr_list):
                continue
            name_block = tr_list[i + 1]

            if name_block is None:
                continue
            
            name_element = name_block.find('b')
            if name_element is None:
                continue
            
            name = name_element.getText()
            terminated = name_block.find('i').getText() 
            
            if "TERMINATED" not in terminated:
                terminated = "ONGOING"
            
            print(name)
            
            tds = name_block.find_all('td')
            rep_info = str(tds[len(tds) - 1]).replace('\n','n')
            # End Name Block
        
        elif not title and which_block == "pending":
            # Get info from pending count
            disposition_block = tr_list[i]
            
            if disposition_block is None:
                continue
            
            if("None" in disposition_block.find('td').getText()):
                pending_counts.append(None)
                pending_counts.append(None)
            else:
                # Append pending count, and then the dispostition of count
                pending_counts.append(disposition_block.find_all('td')[0].getText())
                pending_counts.append(disposition_block.find_all('td')[2].getText())
            
        elif not title and which_block == "terminated":
            # Get info from terminated count
            terminated_block = tr_list[i]
            
            if terminated_block is None:
                continue
                
            if("None" in terminated_block.find('td').getText()):
                terminated_counts.append(None)
                terminated_counts.append(None)
            else:
                # Append terminated count, and then the dispostition of count
                terminated_counts.append(terminated_block.find_all('td')[0].getText())
                terminated_counts.append(terminated_block.find_all('td')[2].getText())
        
        # Set the mode for the next pass through the tr list, and return the previoius defendant 
        if title and "Pending Counts" in underlined_text:
            which_block = "pending"
        elif title and "Terminated Counts" in underlined_text:
            which_block = "terminated"
        elif title and "Highest" in underlined_text:
            which_block = "highest"
        elif title and "Complaints" in underlined_text:
            # Reached final block in defendant
            which_block = "complaints"
            
#             # Tempterarily save the lists of counts
#             counts1 = list(pending_counts)
#             counts2 = list(terminated_counts)
                    
#             # Reset counts
#             pending_counts = []
#             terminated_counts = []
            
            defendant_info = [name, terminated.replace("TERMINATED: ", ""), rep_info]
            
            yield defendant_info, [pending_counts[i] for i in range(len(pending_counts), 2)],
                                   [pending_counts[i+1] for i in range(len(pending_counts), 2)],
                                   [term_counts[i] for i in range(len(term_counts), 2)], 
                                    [term_counts[+ 1] for i in range(len(term_counts), 2)]
                            

In [20]:
import csv
import re

with open('2FLY.csv', "w", newline="") as csvfile:
    writer = csv.writer(csvfile, delimiter=",")
    writer.writerow(["Name", "Terminated", "Representative Info", "Pending Counts", "Months", "Pending Disposition", "Terminated Counts", "Terminated Disposition"])
    
    for defendant_info, pending_counts, disp1, term_counts, disp2 in iterate_through_html():
#         for i in range(0, max(len(pending_counts), len(term_counts)), 2):
#             # Pad with empty columns if necessary
#             if i < len(pending_counts):
                
# #                 print(pending_counts[i+1])
                
#                 num = None
    
#                 if pending_counts[i+1] is not None:
#                     num = re.search('[0-9]+', pending_counts[i+1])
            
#                 a, b, c = pending_counts[i], num, pending_counts[i + 1]
#             else:
#                 a, b, c = None, None, None
                
#             if i < len(term_counts):
#                 d, e = term_counts[i], term_counts[i + 1]
#             else:
#                 d, e = None, None
            
#             if i > 0:
#                 defendant_info = [None, None, None]
            

        num = None
        if len(pending_counts) > 1:
            num = re.search('[0-9]+', pending_counts[1])

        if num is None:
            num = '0'
        else:
            num = num.group()

        row = defendant_info + pending_counts + [str(num)] +  term_counts
      
        
        writer.writerow(row)
        
        

Defendant                                 (1)
Laquan Parrish


ValueError: not enough values to unpack (expected 5, got 3)